In [2]:
import json 
import re
import cv2
import matplotlib.pyplot as plt

In [7]:
def bounding_box_to_segment(image, bb):
    x_coords = [point['x'] for point in bb]
    y_coords = [point['y'] for point in bb]

    # Determine the bounding box
    left = min(x_coords)
    top = min(y_coords)
    right = max(x_coords)
    bottom = max(y_coords)
    return image[top:bottom, left:right]

teeth_found = 0
azure_labels = []
img_i = 0
for i in range(1, 140):
    file_path = f'/home/riikoro/fossil_data/Fort Tenan Catalogue/scan.pdf_pages/page{i}.json'
    pdf = cv2.imread(f'/home/riikoro/fossil_data/Fort Tenan Catalogue/scan.pdf_pages/page{i}.png')
    # pdf = cv2.cvtColor(pdf, cv2.COLOR_BGR2RGB) 
    # Open the JSON file
    with open(file_path, 'r') as json_file:
        # Load the JSON data into a Python object
        data = json.load(json_file)

    for block in data['readResult']['blocks']:
        for line in block['lines']:
            # print(line['text'])
            for i, word in enumerate(line['words']):
                # prior: catalogue has "description" as last column. only find teeth in this column
                if word['boundingPolygon'][0]['x'] > 1225:
                    is_tooth = re.match('^[a-zA-Z]\d$', word['text'])
                    if (is_tooth):
                        teeth_found += 1
                        azure_label = word['text']
                        bounding_box = word['boundingPolygon']
                        azure_labels.append(azure_label)
                        img_segment = bounding_box_to_segment(pdf, bounding_box)
                        cv2.imwrite(f'/home/riikoro/fossil_data/tooth_samples/v3/{img_i}.png', img_segment)
                        img_i += 1

In [5]:
teeth_found

232

In [6]:
azure_labels

['M3',
 'H1',
 'T2',
 'M3',
 'p4',
 'M3',
 'H2',
 'M2',
 'M3',
 'M2',
 'P4',
 'P3',
 'M3',
 'P3',
 'M3',
 'M2',
 'M3',
 'M3',
 'H2',
 'M3',
 'M2',
 'M2',
 'M3',
 'P3',
 'M2',
 'P3',
 'M3',
 'M3',
 'M2',
 'M3',
 'M2',
 'P4',
 'M3',
 'P2',
 'M3',
 'M2',
 'M3',
 'M3',
 'P3',
 'M3',
 'M2',
 'P3',
 'M3',
 'M2',
 'M2',
 'M3',
 'M3',
 'H3',
 'M3',
 'p2',
 'p3',
 'p3',
 'p2',
 'P3',
 'p3',
 'p3',
 'M1',
 'H3',
 'M2',
 'M2',
 'H1',
 'M3',
 'H3',
 'M2',
 'H3',
 'M2',
 'P3',
 'H3',
 'M2',
 'P3',
 'M3',
 'H3',
 'H3',
 'M3',
 'M3',
 'H3',
 'M3',
 'P3',
 'M2',
 'M2',
 'H3',
 'H3',
 'P4',
 'P4',
 'M3',
 'M2',
 'M2',
 'H3',
 'M3',
 'M3',
 'M3',
 'P4',
 'P4',
 'M3',
 'M3',
 'P4',
 'M3',
 'H3',
 'M2',
 'P4',
 'P4',
 'M3',
 'M3',
 'M3',
 'M2',
 'M3',
 'M3',
 'M2',
 'P3',
 'M2',
 'M2',
 'p2',
 'H2',
 'M2',
 'M3',
 'M3',
 'p3',
 'p3',
 'M3',
 'M2',
 'M2',
 'M2',
 'M3',
 'M2',
 'M3',
 'M2',
 'M2',
 'M2',
 'M2',
 'M3',
 'M2',
 'M2',
 'M2',
 'P3',
 'M2',
 'M2',
 'M1',
 'M2',
 'M1',
 'M2',
 'M2',
 'p2',
 'M2',

In [12]:
with open('/home/riikoro/fossil_data/tooth_samples/v3/azure_labels.txt', 'w') as out:
    out.write('\n'.join(azure_labels))